## Classification (b)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

# Load the configuration.
config = load_config("./data")

# Create the model
model  = Neuralnetwork(config)
# Load the data
x_train, y_train = load_data(path="./data", mode="train") # output normalized and one hot encoding
x_test, y_test = load_data(path="./data", mode="test")# output normalized and one hot encoding

# Use 10 examples one from each category

$$$$$$ (3072, 50)
$$$$$$ (50, 50)
$$$$$$ (50, 11)


In [2]:

# Data loading and spliting
x_b, y_b = load_data(path="./data", mode="train")

class_num = list(range(10)) # 10 sample in range
xtrain = []
ytrain = []

parse = 0;
for k in range(5):
    xtrain.extend(x_b[parse :parse+10])
    ytrain.extend(y_b[parse :parse+10])
    parse += 10000
        
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)


In [4]:
# Load model para
config_prob_b = {}
config_prob_b['layer_specs'] = [3072, 50, 50, 11]
config_prob_b['activation'] = 'ReLU'
config_prob_b['learning_rate'] = 0.15 
config_prob_b['batch_size'] = 128 
config_prob_b['epochs'] = 100  
config_prob_b['early_stop'] = True 
config_prob_b['early_stop_epoch'] = 5  
config_prob_b['L2_penalty'] = 0  
config_prob_b['momentum'] = True  
config_prob_b['momentum_gamma'] = 0.9  

model_b = Neuralnetwork(config_prob_b)
model_b.forward(xtrain,ytrain)
model_b.backward()


$$$$$$ (3072, 50)
$$$$$$ (50, 50)
$$$$$$ (50, 11)


array([[ 2.21435121e+001,  1.21220289e+001,  1.94522751e+001, ...,
        -2.43969964e+001, -2.01970159e+001,  5.34331816e+000],
       [-4.35552189e-158,  1.31478795e-157, -1.51723370e-157, ...,
        -1.25322357e-157, -1.03204203e-157,  2.89141019e-157],
       [-1.69037190e+001,  3.92642022e+001, -2.41959006e+001, ...,
        -3.45174435e+001,  2.19288350e+001,  5.97093389e+000],
       ...,
       [ 3.69138279e+000,  5.63699191e+001,  4.37535997e+001, ...,
         1.50553224e+001,  1.52009729e+000,  8.79999837e+001],
       [-2.20527202e+001,  4.95845634e+001,  3.45546500e+001, ...,
        -3.03912400e+001, -3.00965247e+001, -1.40381451e+001],
       [ 6.48280465e+001, -1.82342891e+001,  3.38145742e+001, ...,
        -2.07644852e+001,  8.02769900e+000,  6.35492924e+001]])

In [9]:
# d_b d_w comparison

## Part (b) Estimation of bias weight and weight
def Numerical_Estimation_bias(model, layer, eps, input_idx, output_idx):
    layer.b[0][output_idx] += eps # E(w+e)
    loss_1 = model.forward(np.array(xtrain), np.array(ytrain))[1]
    layer.b[0][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(np.array(xtrain), np.array(ytrain))[1]
    layer.b[0][output_idx] += eps
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

def Numerical_Estimation_weight(model, layer, eps, input_idx, output_idx):
    layer.w[input_idx][output_idx] += eps # E(w+e)
    loss_1 = model.forward(np.array(xtrain), np.array(ytrain))[1]
    layer.w[input_idx][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(np.array(xtrain), np.array(ytrain))[1]
    layer.w[input_idx][output_idx] += eps
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

eps = 10e-2 # difference expectation to 10e-4


db = []
dw = []
db_est = []
dw_est = []
for layer in model_b.layers:
    #d_b_est.append(Numerical_Estimation_bias(model=model_b, layer=layer, eps=eps,input_idx=0, output_idx=1))
    #d_w_est.append([Numerical_Estimation_weight(model=model_b, layer=layer, eps=eps, input_idx=0, output_idx=1), Numerical_Estimation_weight(model=model, layer=layer, eps = eps, input_idx=0, output_idx=2)])
    db.append(layer.d_b[0]) 
    dw.append([layer.d_w[0][0] ,layer.d_w[0][1]])
        
print('db: {}'.format(d_b))
print('db est: {}'.format(d_b_estimate))
print('dw: {}'.format(d_w))
print('dw est: {}'.format( d_w_estimate))

AttributeError: 'Activation' object has no attribute 'd_b'

In [21]:
np.random.seed(42)
w = np.random.randn(50, 3072)
x = xtrain

k = np.dot(x,w.T)
np.shape(k)


(50, 50)